In [1]:
import io
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [4]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
for i, data in enumerate([train_data, test_data]):
    data.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)
    data["Sex"] = data["Sex"].replace(['male', 'female'], [1, 0])
    data['Embarked'] = data["Embarked"].replace(['S', 'C', 'Q'], [0, 1, 2])

In [5]:
male = train_data.Sex == 1
female = train_data.Sex == 0
test_male = test_data.Sex == 1
test_female = test_data.Sex == 0

mean_male_age = np.round(train_data.loc[male, ['Age']].mean())
mean_female_age = np.round(train_data.loc[female, ['Age']].mean())
embarked_mode = train_data.Embarked.value_counts().keys()[0]

In [6]:
train_data.loc[male, ['Age']] = train_data.loc[male, ['Age']].fillna(mean_male_age)
train_data.loc[female, ['Age']] = train_data.loc[female, ['Age']].fillna(mean_female_age)

test_data.loc[test_male, ['Age']] = test_data.loc[test_male, ['Age']].fillna(mean_male_age)
test_data.loc[test_female, ['Age']] = test_data.loc[test_female, ['Age']].fillna(mean_female_age)
train_data[['Embarked']] = train_data[["Embarked"]].fillna(embarked_mode)
test_data.Fare = test_data.Fare.fillna(train_data.Fare.mean())

train_data.Survived.replace(0, -1, inplace=True)

train_data["Relatives"] = train_data.SibSp + train_data.Parch
test_data["Relatives"] = test_data.SibSp + test_data.Parch

train_data["RelativesCat"] = pd.cut(train_data.Relatives.values, bins=[-np.inf, 0.7, 3.7, np.inf], labels=[0, 1, 2])
test_data["RelativesCat"] = pd.cut(test_data.Relatives.values, bins=[-np.inf, 0.7, 3.7, np.inf], labels=[0, 1, 2])

train_data["AgeCat"] = pd.cut(train_data.Age.values, bins=[-np.inf, 20.315, 40.21, 60.105, np.inf], labels=[0, 1, 2, 3])
test_data["AgeCat"] = pd.cut(test_data.Age.values, bins=[-np.inf, 20.315, 40.21, 60.105, np.inf], labels=[0, 1, 2, 3])

test_data["Survived"] = 0

In [7]:
col = ["Pclass", "Sex", "Embarked", "RelativesCat", "AgeCat", "Survived"]

# KNN
## 0.77272 Kaggle


In [ ]:
def distance(test_point, train_point):
    return np.sum(np.abs(test_point - train_point))

In [ ]:
rnd_indices = np.random.randint(0, train_data.shape[0], 100)
knn_train = train_data.loc[set(range(train_data.shape[0])) - set(rnd_indices), col]
knn_val = train_data.loc[rnd_indices, col]
knn_train.reset_index(inplace=True)
knn_val.reset_index(inplace=True)

In [ ]:
knn_val['pred'] = 0

In [ ]:
tmp = np.zeros(knn_train.shape[0])
for k in [11, 9, 7, 5, 4, 3, 2]:
    for i, val in enumerate(knn_val.values[:, 1:-2]):
        res = 0
        for idx, train in enumerate(knn_train.values[:, 1:-1]):
            tmp[idx] = distance(val, train)
        mx = np.max(tmp) + 1
        for _ in range(k):
            mn_idx = np.argmin(tmp)
            res += knn_train.values[mn_idx, -1]
            tmp[mn_idx] = mx
        knn_val.iloc[i, -1] = 1 if res > 0 else -1
    print(k, np.sum(knn_val.Survived == knn_val.pred))

11 79
9 80
7 81
5 82
4 81
3 78
2 74


In [ ]:
knn_test = test_data.loc[:, col[:-1]]
knn_test

,Pclass,Sex,Embarked,RelativesCat,AgeCat
0,3,1,2,0,1
1,3,0,0,1,2
2,2,1,2,0,3
3,3,1,0,0,1
4,3,0,0,1,1
...,...,...,...,...,...
413,3,1,0,0,1
414,1,0,1,0,1
415,3,1,0,0,1
416,3,1,0,0,1


In [ ]:
train_data.loc[:, col].shape, knn_test.shape

((891, 6), (418, 5))

In [ ]:
tmp = np.zeros(train_data.shape[0])
k = 5
for i, test in enumerate(knn_test.values):
    res = 0
    for idx, train in enumerate(train_data.loc[:, col[:-1]].values):
        tmp[idx] = distance(test, train)
    mx = tmp.max() + 1
    for _ in range(k):
        mn_idx = np.argmin(tmp)
        res += train_data.iloc[mn_idx, 1]
        tmp[mn_idx] = mx
    if res > 0:
        test_data.iloc[i, -1] = 1

In [ ]:
test_data[["PassengerId",'Survived']].to_csv('5nn_titanic.csv', index=False)

# Logistic Regression
## 0.74641 reg Kaggle
## 0.76555 no reg Kaggle

In [12]:
from sklearn.model_selection import train_test_split

In [11]:
logistic_data = train_data.loc[:, col].copy()
logistic_data

,Pclass,Sex,Embarked,RelativesCat,AgeCat,Survived
0,3,1,0.0,1,1,-1
1,1,0,1.0,1,1,1
2,3,0,0.0,0,1,1
3,1,0,0.0,1,1,1
4,3,1,0.0,0,1,-1
...,...,...,...,...,...,...
886,2,1,0.0,0,1,-1
887,1,0,0.0,0,0,1
888,3,0,0.0,1,1,-1
889,1,1,1.0,0,1,1


In [ ]:
X = logistic_data.iloc[:, :-1].values
y = logistic_data.iloc[:, [-1]].values

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# X_train_B = np.c_[np.ones((X_train.shape[0], 1)), X_train]
# X_val_B = np.c_[np.ones((X_val.shape[0], 1)), X_val]
X_B = np.c_[np.ones((X.shape[0])), X]

In [ ]:
def sigmoid(x_w, y):
    return 1 / (1 + np.exp(-y * x_w))

In [ ]:
epochs = 3000
eta0 = 100
weights = np.random.randn(X_B.shape[1], 1)
errors = []

for epoch in range(1, epochs + 1):
    eta = eta0 / (epoch + 10)
    vari = np.var(weights) ** 2
    grad = np.zeros((X_B.shape[1], 1))
    log_loss = 0
    for point, target in zip(X_B, y):
        log_loss += np.log(1 + np.exp(-target * point.dot(weights)))# + weights.T.dot(weights) / 2 / vari
        grad += ((1 - sigmoid(point.dot(weights), target)) * point.reshape(-1, 1) * (-target)) #+ weights / vari)
    weights -= eta * grad / X_B.shape[0]
    print("EPOCH: {} --- LOG_LOSS: {}".format(epoch, log_loss / X_B.shape[0]))

In [ ]:
pred = np.round(sigmoid(X_B.dot(weights), np.ones((X_B.shape[0], 1))))
pred[pred == 0] = -1

In [ ]:
np.sum(pred == y) / pred.shape[0]

0.8024691358024691

In [ ]:
X_test = test_data.loc[:, col[:-1]].values
X_test_B = np.c_[np.ones((X_test.shape[0], 1)), X_test]

In [ ]:
y = np.round(sigmoid(X_test_B.dot(weights), np.ones((X_test.shape[0], 1))))

In [ ]:
y = np.c_[test_data.PassengerId.values, y]

In [ ]:
log_test = pd.DataFrame(y, columns=["PassengerId", "Survived"])
log_test = log_test.astype(np.int64)

In [ ]:
log_test.to_csv("log_titanic_no_reg.csv", index=False)

# SVM
## 0.76555 no reg Kaggle

In [151]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked', 'Relatives', 'RelativesCat', 'AgeCat'],
      dtype='object')

In [8]:
svm_col = ["Pclass", "Sex", "Embarked", "SibSp", "Parch", "AgeCat", "Survived"]

In [9]:
X = train_data.loc[:, svm_col[:-1]].values
X_B = np.c_[np.ones((X.shape[0], 1)), X]
y = train_data.loc[:, [svm_col[-1]]].values

In [154]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_B = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_val_B = np.c_[np.ones((X_val.shape[0], 1)), X_val]

In [ ]:
epochs = 3000
a = 1
weights = np.random.randn(X_B.shape[1], 1)
best_weights = np.zeros((X_B.shape[1], 1))
best_val = np.inf
for epoch in range(epochs):
    #TRAIN
    hinge_loss = 0
    z = np.zeros((X_B.shape[1], 1))
    grad = np.zeros((X_B.shape[1], 1))
    for point, target in zip(X_B, y):
        hinge_loss += max(0, 1 - target * (point.dot(weights)))
        grad += (- target * point.reshape((-1, 1))) if target * (point.dot(weights)) < 1 else 0
        z += grad ** 2
    hinge_loss /= X_B.shape[0]
    grad /= X_B.shape[0]
    z /= X_B.shape[0]
    weights -= a * grad / np.sqrt(z + 1e-5)

    #VAL
    # tmp = 1 - X_val_B.dot(weights)
    # val_hinge = np.sum(np.c_[np.zeros((tmp.shape[0], 1)), tmp].max(axis=1)) / tmp.shape[0]
    print("EPOCH: {} --- HINGE_LOSS: {} ---".format(epoch, hinge_loss))

    # if val_hinge < best_val:
    #     best_val = val_hinge
    #     best_weights = weights.copy()

In [20]:
# print("Train weights: ", np.sum(np.sign(X_train_B.dot(weights)) == y_train) / y_train.shape[0])
# print("Train best_weights: ", np.sum(np.sign(X_train_B.dot(best_weights)) == y_train)/ y_train.shape[0])
# print("Val weights: ", np.sum(np.sign(X_val_B.dot(weights)) == y_val)/ y_val.shape[0])
# print("Val best_weights: ", np.sum(np.sign(X_val_B.dot(best_weights)) == y_val)/ y_val.shape[0])
print("X_B weights: ", np.sum(np.sign(X_B.dot(weights)) == y)/ y.shape[0])
#print("X_B best_weights: ", np.sum(np.sign(X_B.dot(best_weights)) == y)/ y.shape[0])

X_B weights:  0.7867564534231201


In [144]:
X_test = test_data.loc[:, col[:-1]].values
X_test_B = np.c_[np.ones((X_test.shape[0], 1)), X_test]
y_pred = np.sign(X_test_B.dot(weights))
y_pred[y_pred == -1] = 0
y_pred = np.c_[test_data.PassengerId.values, y_pred]

In [148]:
svm_test = pd.DataFrame(y_pred, columns=["PassengerId", "Survived"])
svm_test = svm_test.astype(np.int64)
svm_test

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [150]:
svm_test.to_csv("svm_titanic_no_reg.csv", index=False)